# Feature engineering

## Introduction
We always look at a round from the point of view of team_1.

## Features: 
- **target**: team_1 won the round
- **kills**: Amount of players killed by team_1
- **deaths**: Amount of players that died from team_1
- **equipment_value_ratio**: Total equipment value compared to equipment value of team_2. *This is better than saving the equipment values from both the teams. $650 in a pistol round is normal. $650 in round 10 can probably be considered as a saving round.*
- **first_kill**: Did team_1 make the first kill. Making the first kill results in a higher chance of winning the round.

# Features to implement
- **defuse_kit_count**: Amount of defusekits. 
  - **values**: [None,0,1,2,3,4,5] (None when team_1 are terrorists)
- **alive_ratio_on_bomb_planted**: Amount of players alive from team_1 compared to players alive from team_2 when the bomb is planted.
- **distance_from_bomb_on_plant**: Average distance from team_1 when bomb is planted. (Maybe they are already saving)




In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [9]:
# Feature engineering
from feature_engineering import db
from feature_engineering import parser
from dataclasses import make_dataclass,fields
from bson.objectid import ObjectId
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import minmax_scale

collections = db.get_collections()
Round = make_dataclass("Round", [("kills", int),("deaths", int),("first_blood", int),("round_winstreak", int)])

parsed_rounds = []
target = []
matches = collections["matches"].find({}).limit(1)


for match in matches:
  round_winstreak = 0
  team_1_id = match["teams"][0]

  # ROUND
  for round_id in match["rounds"]:

    # Check if round exists
    if collections["rounds"].find_one({ "_id": ObjectId(round_id)}) is None:
      continue

    round = parser.Round(round_id,team_1_id)
    (kills, deaths) = round.kills_and_deaths()

    if round.is_win() :
      target.append(1)
      round_winstreak+= 1
    else:
      target.append(0)
      round_winstreak=0

    parsed_rounds.append(Round(kills=kills,deaths=deaths,first_blood=round.is_first_blood(), round_winstreak = round_winstreak))

print("Parsed rounds:", len(parsed_rounds))
data = pd.DataFrame(parsed_rounds)



Parsed rounds: 30


In [19]:
# We should how diverse our dataset is. If in 80% of our games team_1 wins.
# We will learn the model that it can always pick team_1 and get an accuracy of 80%. We don't want that :)

# Split the data in training and testing data
train_X,test_X, train_Y, test_Y = train_test_split(data,target,test_size=0.2, random_state=42)



# Hyperparameter tuning
With SVM we try to find the maximum margin separator between our two classes (team_1_wins, team_2_wins). This is the line the furtest from the nearest training data points. SVM calculates the distance to the closest datapoint for each possible line and picks the one with the highest distance. This makes an SVM a **maximum margin estimator. 

In most real probelms, it is not possible to find the perfect separating plane. Sometimes there are datapoints that are not closer to the other class. To handle this we allow the SVM to soften the margin. Which means we allow some of the points to creep into the margin if that allows a better fit. This transforms our SVM into a soft-margin classifier since we are allowing for a few mistakes. This is typically called C.

C is a hyperparameter that needs to be tuned based on the dataset.

In [20]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = { "kernel": ["rbf", "linear", "sigmoid"], "C": range(1,50)}
clf = GridSearchCV(svm.SVC(), parameters)
clf.fit(train_X, train_Y)

print(clf.cv_results_)

# TODO: handle overfitting, with cross validation and folding

{'mean_fit_time': array([0.00220819, 0.0012115 , 0.00123787, 0.00132589, 0.00154986,
       0.00176582, 0.00157566, 0.00143147, 0.00147333, 0.00128088,
       0.00123096, 0.00115118, 0.00113769, 0.0012157 , 0.00135016,
       0.00140662, 0.00115099, 0.0011343 , 0.00127907, 0.00140271,
       0.00116911, 0.00126424, 0.00116768, 0.0013135 , 0.00138488,
       0.00139008, 0.00130105, 0.00115328, 0.00126309, 0.00135283,
       0.00123591, 0.00118618, 0.00112829, 0.00113707, 0.00125613,
       0.00127192, 0.0011622 , 0.0012001 , 0.00114665, 0.00117373,
       0.00123019, 0.00121107, 0.00112801, 0.00113645, 0.00119061,
       0.00125928, 0.00134754, 0.00133452, 0.00122886, 0.00118709,
       0.00132031, 0.00128927, 0.00124593, 0.00126185, 0.00120659,
       0.0012188 , 0.0012939 , 0.00129857, 0.00119004, 0.00113635,
       0.00120335, 0.00148253, 0.00123358, 0.00118155, 0.0011796 ,
       0.0011652 , 0.0012084 , 0.00124116, 0.00122709, 0.00133328,
       0.00111718, 0.0012846 , 0.00123725, 0

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix
# Performance metrics

prediction = clf.predict(test_X)
accuracy = accuracy_score(test_Y, prediction)
print(accuracy)

confusion_matrix(test_Y, prediction)


AttributeError: predict_proba is not available when  probability=False

In [ ]:
# Hyper parameter Tuning





SyntaxError: unexpected EOF while parsing (1183308413.py, line 34)

# Live demo parsing

A second part, we unfortunately had no time to implement, is **live demo parsing**.

We need to start by creating a probability N by N matrix, with N the different states.
This is also called a ** stochastic matrix**. Each combination of a row(x) and a column(y) is the probability to go from stateX to stateY.

Since we eventually want to find the winner of a game we also need something called absorbing states. **An absorbing state is a state that, once entered, cannot be left**
